# Interactive Malthus Economy

In the first part of the project the **basic Malthus model** is implemented with interactive graphs, so it's possible to add shocks to the emount of land (X) and the technology level (A). In the second part of the project the model is **extended to include technological growth**.

**Time:** Discrete and represented by $ t \in {0,1,...,\infty} $

**Production function:** Cobb-Douglas
$$ Y_t = L_t^{1-\alpha}(AX)^\alpha $$
$A$ and $X$ are independent of time. $A$ is the technology-level and $X$ is the amount of land available.

The production function can also be written as production pr. capita, $y_t$, which is found by dividing the equation above by $L_t$:
$$ y_t = \frac{Y_t}{L_t} = (\frac{AX}{L_t})^\alpha $$

**Fertility curve:**
$$n_t = \frac{1-\beta}{\lambda}y_t (1-\tau)$$

In the equation for the fertility-curve the expression $1-\beta$ changes with preferences/culture about having children, $\lambda$ represents the lower costs of having children and $\tau$ represents taxes in society.


**Transition-equation:**
$$L_{t+1} = n_t L_t + (1-\mu) L_t$$
In the transition equation above $L_0$ is given.

A key assumption in the Malthus economy is that the birth rate, $n_t$, rises with income pr. capita, $y_t$, such that in steady state the birth rate is going to be: $$ n^* = \eta * y^* $$ $\eta$ is a positive parameter.

## Analytical solution to the Malthus model

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details.

In [1]:
# Importing packages
import sympy as sm                                  # Symbolic python
import ipywidgets as widgets                        # Widgets
import seaborn as sns                               # Seaborn for graphs
import matplotlib.pyplot as plt                     # Pyplot for plots
from IPython.display import display, Markdown       # Display and markdown

# Importing ModelClass
import ModelClass

In [2]:
model = ModelClass.MalthusModel()

transition_eq = model.symbolic_L()

latex_with_code = r"$ \Large{ L_{t+1} = " + sm.latex(transition_eq) + r"}$"

print("To derieve the analytical steady state the transition equation describing the change in labor force will be used:")
display(Markdown(latex_with_code))

To derieve the analytical steady state the transition equation describing the change in labor force will be used:


$ \Large{ L_{t+1} = L_{t} \left(1 - \mu\right) + L_{t}^{1 - \alpha} \eta \left(A X\right)^{\alpha}}$

In [3]:
# Steady state expression for labor force
L_ss_symbolic = model.symbolic_ss_L()

latex_with_code = r"$ \Large{ L^{*} = " + sm.latex(L_ss_symbolic) + r"}$"

print("Steady state expression for the labor force:")
display(Markdown(latex_with_code))


# Steady state expression for output pr. worker
y_ss_symbolic = model.symbolic_ss_y()

latex_with_code = r"$ \Large{ y^{*} = " + sm.latex(y_ss_symbolic) + r"}$"

print("Steady state expression for output pr. worker:")
display(Markdown(latex_with_code))


Steady state expression for the labor force:


$ \Large{ L^{*} = \left(\frac{\eta \left(A X\right)^{\alpha}}{\mu}\right)^{\frac{1}{\alpha}}}$

Steady state expression for output pr. worker:


$ \Large{ y^{*} = \left(A X \left(\frac{\eta \left(A X\right)^{\alpha}}{\mu}\right)^{- \frac{1}{\alpha}}\right)^{\alpha}}$

It's possible to simplify the expression for $L^*$ and $y^*$ a lot more than what has been done in the above. Something with symbolic python is not very good at simplifying "complex" expressions.

### Turning the symbolic math steady state expressions into python functions using lambdify

In [4]:
# Labor force in steady state
labor_force_ss_func = model.symbolic_ss_L_lambdify()

values = model.val

labor_force_ss = labor_force_ss_func(values.technology, values.land, values.alpha, values.mu, values.eta)

print("State state labor force:", format(labor_force_ss, ".10f"))

State state labor force: 1256.7396638226


In [5]:
# Output pr. worker in steady state
output_pr_worker_ss_func = model.symbolic_ss_y_lambdify()

values = model.val

output_pr_worker_ss = output_pr_worker_ss_func(values.technology, values.land, values.alpha, values.mu, values.eta)

print("State state output pr. worker:", format(output_pr_worker_ss, ".10f"))

State state output pr. worker: 0.3428571429


## Numerical solution to the Malthus model

To begin with we find the amount of labor force, $L^*$, in the steady state.

In the code below we have used **optimize.root** in the scipy to find the optimal value of $L_t$ where the difference between $L_t$ and $L_{t+1}$ is equal to zero. In that scenario we have found the steady state value of $L^*$. 

The method used in the optimize.root function is hybr. This builds upon the Powell Method of optimization. The method is more efficient than many other approaches using a "steepest gradient" approach since it requires fewer steps to converge to minima/maxima for continous functions that are near-quadratic at their minima/maxima. It is also especially fool-proof against non-differentiable, continous functions as the optimization method does not use any derivatives in its search for extremeties.

In the **ModelClass.py** the parameters for the model have been defined, as well as the functions for the model.

The model doesn't have any problem with convergence, but that is not surprising since it doesn't have multiple local minima which could "confuse" the optimization algorithm and since our functions are continous. However the optimization function **numerical_solution_steady_state** in the ModelClass has been implemented with **multi start**, so it should also be possible to find the global minima in case anyone wants to update the model where the optimization could be harder for the algorithm e.g. the case where there are multiple local minima.

In [6]:
model = ModelClass.MalthusModel()

(labor_A_ss, labor_ss, output_ss, output_pr_worker_ss, birth_rate_ss, residual_ss) = model.numerical_solution_steady_state()

print("Steady state value for labor force:", format(labor_ss[0], ".10f"))
print("Steady state value for output:", format(output_ss[0], ".10f"))
print("Steady state value for output pr. worker:", format(output_pr_worker_ss[0], ".10f"))
print("Steady state value for birth rate:", format(birth_rate_ss[0], ".10f"))

Steady state value for labor force: 1256.7396638226
Steady state value for output: 430.8821704535
Steady state value for output pr. worker: 0.3428571429
Steady state value for birth rate: 0.4500000000


### Comparing the analytical and numerical results in steady state

It can be seen that the steady state values are the same when using either the symbolic or the numerical solution to the Malthus model.

## The model visualized towards steady state

In the code following below, we visualize the Basic Malthus Model's convergence towards a steady state. Furthermore, we add the opportunity to impose positive or negative shocks to the production factors, A & X, at a specified time, while also adding the opportunity to change parameter values in the model.

Generally, it is seen that for identical parameter values, shocks to A and X will lead to higher output and population. Initially, the output pr. worker increases, but as time passes, this higher output pr. worker translates into higher birth rates "crowding out" the  increased output. After a short transition period, shocks to A and X does not have any effect on the output pr. worker but only increases the total population and output in a linear relationship. 

In [7]:
# Function to update the transition to steady state
def update_transition_plots(alpha, beta, small_lambda, tau, mu, X_shock_size, A_shock_size, X_shock_time, A_shock_time):

    # Simulate values towards steady state
    (L, Y, y, X, A, n, _) = model.simulate_transition_ss(False, True, model.val.g, alpha, beta, small_lambda, tau, mu, X_shock_size, A_shock_size, X_shock_time, A_shock_time)

    model.plot_transition_towards_ss(L, Y, y, n, X, A)

(alpha_slider, beta_slider, small_lambda_slider, tau_slider, mu_slider) = model.model_parameter_sliders()
(X_shock_size_slider, X_shock_time_slider, A_shock_size_slider, A_shock_time_slider) = model.model_shock_sliders()

display(Markdown("### By using the slider below it's both possible to adjust the parameters in the Malthus economy and add shocks to both the amount of land (X) and the technology level (A) at different times."))

# Display slider widgets and plot
widgets.interactive(update_transition_plots, 
                    # Parameter sliders
                    alpha=alpha_slider, 
                    beta=beta_slider, 
                    small_lambda=small_lambda_slider, 
                    tau=tau_slider, 
                    mu=mu_slider, 
                    # Shock sliders
                    X_shock_size = X_shock_size_slider, 
                    X_shock_time = X_shock_time_slider,
                    A_shock_size = A_shock_size_slider,  
                    A_shock_time = A_shock_time_slider)


### By using the slider below it's both possible to adjust the parameters in the Malthus economy and add shocks to both the amount of land (X) and the technology level (A) at different times.

interactive(children=(FloatSlider(value=0.15, continuous_update=False, description='Alpha', max=1.0, step=0.01…

## Malthus model extended with technological growth

In the basic Malthus model it assumed that there is no technological growth. However the model can be extended to include technological growth, as Carl-Johan Dalgaard has done in a note called "A simple dynamic Malthusian model". When there is technological growth in the economy it means that $A_{t} \neq A_{t+1}$, but that there is a growth factor $\frac{A_{t+1}}{A_t}=g$.

Since technological growth leads to higher output ($Y_t$) which leads to a larger population ($L_t$) it means that $L_t$ will not converge to a steady state level. Instead a steady state in the modified model can be found by looking at:

$$ l_t = \frac{L_t}{A_t} $$

The above expression is the labor force in relation to the technological level in the economy.

In this model project we're not going to derive the law of motion for the modified model, but note that the law of motion is the following:

$$ l_{t+1} = \eta * g^{-1} * l_{t}^{1-\alpha} * X^{\alpha} + g^{-1}*(1-\mu)*l_t $$

Where $l_0$ is given.

### Steady state with technological growth

In the next section the steady state of the model with technological growth is found, by numerical analysis. It's almost the same optimiztion as in the model without technological growth apart from the fact that the transition equation is now $l_{t+1}$ instead of $L_{t+1}$. 

In [8]:
model_with_growth = ModelClass.MalthusModel()

(labor_A_ss, labor_ss, output_ss, output_pr_worker_ss, birth_rate_ss, residual_ss) = model.numerical_solution_steady_state(with_A_growth = True)

print("Steady state value for labor force adjusted for technological level:", format(labor_A_ss[0], ".10f"))
print("Steady state value for output pr. worker:", format(output_pr_worker_ss[0], ".10f"))
print("Steady state value for birth rate:", format(birth_rate_ss[0], ".10f"))

Steady state value for labor force adjusted for technological level: 940.4658444849
Steady state value for output pr. worker: 0.3580952381
Steady state value for birth rate: 0.4700000000


We have that the steady state equilibrium in the modified model is $l_{t+1}=l_t=l^*$.

In the code below there has been implemented a slider - in this case only for technological growth - whereby it's possible to see how the economy changes in a state with 1) decreasing technological ($g<1$), 2) unchanged technological level ($g=1$) and 3) technological growth ($g>1$).

In [9]:
# Access model values
val = model.val

# Function to update the transition to steady state
def update_transition_plots_with_g(g):

    # Simulate values towards steady state
    (L, Y, y, _, A, n, l) = model.simulate_transition_ss(True, False, g, val.alpha, val.beta, val.small_lambda, val.tau, val.mu, 0, 0, 0, 0)

    model.plot_transition_towards_ss_with_g_growth(l, A, L, Y, n, y)


# Sliders to adjust the amount of technological growth
g_slider = widgets.FloatSlider(min=0.95, max=1.05, step=0.005, value=model.val.g, description='Technological growth', continuous_update=False)

g_slider.style = {'description_width': 'initial'} # Set the width to 'initial' (default) or a custom value
g_slider.layout.width = '50%'  # Adjust the width of the slider (e.g., '50%', '500px', etc.)

# Display slider widgets and plot
widgets.interactive(update_transition_plots_with_g, g=g_slider)

interactive(children=(FloatSlider(value=1.02, continuous_update=False, description='Technological growth', lay…

Thereby it can be seen that technological growth ($g>1$) will lead to an ever increasing labor force ($L_t$) which will lead to an ever increasing output ($Y_t$), but that the output pr. worker ($y_t$) will remain mostly unchanged in the long run, since increases in output is just offset by increases in the population.

### Conclusion
In this project we have solved and visualized the basic and technological-growth-extended Malthus-model.

Initially, we solved the basic model analytically using the Sympy-module where we lambdified the symbolic math functions into python functions to be solved with specified parameter values.

Afterwards, we solved the model numerically using the optimize.root command from Scipy using an efficient conjugate direction optimizer with fewer calculation steps in comparison to a 'steepest gradient' optimizer. In this part, we showed that we could find the same solution to the model both numerically and analytically. 

We visualized the basic model's convergence with implementations of shocks to production factors at arbitrary times, and/or with alterations of parameter values. The goal of this visualization was to help the reader understand the model's dynamics and the effect of individual parameters - something that may be lost in many economic models' sea of equations.

Lastly, the model was extended with technological growth following Carl-Johan Dalgaard's note "A simple dynamic Malthusian Model". The main point of this section was to visualize how the Malthusian model's output pr. worker always converges to a fixed value, even when both population and output goes towards infinity.

This project's main goal has been to clarify the Malthus-model's dynamics helping future students gain a better understanding of one of the many economic models that they will face in their time as university students.